### TODO list 
- first implement a visualization tool for the feature representation
    - build the VGG net
    - build the NN with white noise as an input
    - perform backpropagation with the formula given in the paper
    - DO NOT implement yet the style stuff